In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
import kaggle

In [5]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [01:08<00:00, 92.6MB/s]
100% 2.29G/2.29G [01:08<00:00, 35.9MB/s]


In [6]:
import zipfile
zip_ref = zipfile.ZipFile("/content/chest-xray-pneumonia.zip","r")
zip_ref.extractall("/content/")
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121, ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import cv2
import os

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS_FEATURE_EXTRACTION = 10
EPOCHS_FINE_TUNING = 15
INITIAL_LR = 1e-3
FINE_TUNE_LR = 1e-5
DRIVE_PATH = '/content/drive/MyDrive/Colab Notebooks/medicalimagefiles'
os.makedirs(DRIVE_PATH, exist_ok=True)


In [ ]:
TRAIN_DIR = '/content/chest_xray/train'
VAL_DIR = '/content/chest_xray/val'
TEST_DIR = '/content/chest_xray/test'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
val_test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)
val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
class_counts = np.bincount(train_generator.classes)
total = sum(class_counts)
class_weight = {
    0: total / (2 * class_counts[0]),  # NORMAL
    1: total / (2 * class_counts[1])   # PNEUMONIA
}

print(f"\nClass distribution in training:")
print(f"  NORMAL: {class_counts[0]} images")
print(f"  PNEUMONIA: {class_counts[1]} images")
print(f"Class weights: {class_weight}")


Class distribution in training:
  NORMAL: 1341 images
  PNEUMONIA: 3875 images
Class weights: {0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}


In [ ]:
def build_model(base_model_builder, model_name, input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    """
    Build a transfer learning model with custom head

    Args:
        base_model_builder: Function to create base model (e.g., DenseNet121)
        model_name: Name for the model
        input_shape: Input image shape

    Returns:
        Compiled model
    """
    # Load pretrained base (ImageNet weights)
    base_model = base_model_builder(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Freeze base model for feature extraction
    base_model.trainable = False

    # Build custom classifier head
    inputs = keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs, name=model_name)

    return model, base_model


In [ ]:
def train_model(model, base_model, model_name):
    """
    Two-stage training: Feature extraction then fine-tuning
    """
    print(f"\n{'='*60}")
    print(f"TRAINING: {model_name}")
    print(f"{'='*60}")

    # ========== STAGE 1: FEATURE EXTRACTION ==========
    print(f"\n🔹 STAGE 1: Feature Extraction (base frozen)")
    print(f"Base model trainable: {base_model.trainable}")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=INITIAL_LR),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )

    callbacks_stage1 = [
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            os.path.join(DRIVE_PATH, f'{model_name}_feature_extraction.h5'), ...
            monitor='val_auc',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]

    history_stage1 = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS_FEATURE_EXTRACTION,
        class_weight=class_weight,
        callbacks=callbacks_stage1,
        verbose=1
    )

    # ========== STAGE 2: FINE-TUNING ==========
    print(f"\n🔹 STAGE 2: Fine-Tuning (unfreeze top layers)")

    # Unfreeze the base model
    base_model.trainable = True

    # Freeze all layers except the top 50
    total_layers = len(base_model.layers)
    freeze_until = total_layers - 50

    for layer in base_model.layers[:freeze_until]:
        layer.trainable = False

    trainable_count = sum([1 for layer in base_model.layers if layer.trainable])
    print(f"Total layers: {total_layers}")
    print(f"Trainable layers: {trainable_count}")

    # Recompile with lower learning rate
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )

    callbacks_stage2 = [
        EarlyStopping(
            monitor='val_loss',
            patience=7,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-8,
            verbose=1
        ),
        ModelCheckpoint(
            os.path.join(DRIVE_PATH, f'{model_name}_fine_tuned.h5'), ...
            monitor='val_auc',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]

    history_stage2 = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS_FINE_TUNING,
        class_weight=class_weight,
        callbacks=callbacks_stage2,
        verbose=1
    )

    print(f"\n✅ {model_name} training completed!")

    return history_stage1, history_stage2

# =================

In [ ]:
def evaluate_model(model, model_name, test_generator):
    """
    Comprehensive evaluation with multiple metrics
    """
    print(f"\n{'='*60}")
    print(f"EVALUATION: {model_name}")
    print(f"{'='*60}")

    # Get predictions
    test_generator.reset()
    y_pred_probs = model.predict(test_generator, verbose=1)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()
    y_true = test_generator.classes

    # Calculate metrics
    test_loss, test_acc, test_precision, test_recall, test_auc = model.evaluate(
        test_generator, verbose=0
    )

    f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall + 1e-7)

    print(f"\n📊 Test Metrics:")
    print(f"  Accuracy:  {test_acc:.4f}")
    print(f"  Precision: {test_precision:.4f}")
    print(f"  Recall:    {test_recall:.4f}")
    print(f"  F1-Score:  {f1_score:.4f}")
    print(f"  AUC:       {test_auc:.4f}")

    # Classification report
    print(f"\n📋 Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['NORMAL', 'PNEUMONIA']))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['NORMAL', 'PNEUMONIA'],
                yticklabels=['NORMAL', 'PNEUMONIA'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(DRIVE_PATH, f'{model_name}_confusion_matrix.png'))
    plt.close()

    # ROC curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_probs)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {test_auc:.3f})', linewidth=2)
    plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {model_name}')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.savefig(os.path.join(DRIVE_PATH, f'{model_name}_roc_curve.png'))
    plt.close()

    return {
        'accuracy': test_acc,
        'precision': test_precision,
        'recall': test_recall,
        'f1_score': f1_score,
        'auc': test_auc
    }

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Generate Grad-CAM heatmap for model interpretability
    """
    # Create a model that maps input to last conv layer and output
    grad_model = Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute gradient
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Gradient of output with respect to conv layer
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy()

In [ ]:
def visualize_gradcam(img_path, model, model_name, last_conv_layer_name):
    """
    Visualize Grad-CAM on an X-ray image
    """
    # Load and preprocess image
    img = keras.preprocessing.image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Get prediction
    pred = model.predict(img_array, verbose=0)[0][0]
    pred_class = "PNEUMONIA" if pred > 0.5 else "NORMAL"

    # Generate heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    # Superimpose heatmap on image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    heatmap = cv2.resize(heatmap, (IMG_SIZE, IMG_SIZE))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original X-Ray')
    axes[0].axis('off')

    axes[1].imshow(heatmap)
    axes[1].set_title('Grad-CAM Heatmap')
    axes[1].axis('off')

    axes[2].imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
    axes[2].set_title(f'Overlay\nPrediction: {pred_class} ({pred:.2f})')
    axes[2].axis('off')

    plt.suptitle(f'{model_name} - Grad-CAM Visualization', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(DRIVE_PATH, f'{model_name}_gradcam.png'))
    print(f"✅ Grad-CAM saved to Drive: {DRIVE_PATH}{model_name}_gradcam.png")
    plt.close()

    print(f"✅ Grad-CAM saved: {model_name}_gradcam.png")
    return pred, pred_class


In [ ]:
def ensemble_predict(models_dict, img_path):
    """
    Ensemble prediction using all models
    """
    print(f"\n{'='*60}")
    print("ENSEMBLE PREDICTION")
    print(f"{'='*60}")

    # Load and preprocess
    img = keras.preprocessing.image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = {}
    for name, model in models_dict.items():
        pred = model.predict(img_array, verbose=0)[0][0]
        pred_class = "PNEUMONIA" if pred > 0.5 else "NORMAL"
        predictions[name] = pred
        print(f"  {name:20s}: {pred:.4f} ({pred_class})")

    # Ensemble (average)
    ensemble_pred = np.mean(list(predictions.values()))
    ensemble_class = "PNEUMONIA" if ensemble_pred > 0.5 else "NORMAL"
    confidence = ensemble_pred * 100 if ensemble_pred > 0.5 else (1 - ensemble_pred) * 100

    print(f"\n🔮 Ensemble Result: {ensemble_class}")
    print(f"   Confidence: {confidence:.2f}%")

    return predictions, ensemble_pred, ensemble_class, confidence


In [ ]:
if __name__ == "__main__":

    # Define models to train
    models_config = [
        (DenseNet121, "DenseNet121", "conv5_block16_concat"),
        (ResNet50, "ResNet50", "conv5_block3_out"),
        (EfficientNetB0, "EfficientNetB0", "top_activation")
    ]

    trained_models = {}
    all_results = {}

    # Train each model
    for base_builder, name, last_conv_layer in models_config:
        print(f"\n\n{'#'*60}")
        print(f"# PROCESSING: {name}")
        print(f"{'#'*60}\n")

        # Build model
        model, base_model = build_model(base_builder, name)
        print(f"✅ Model built: {name}")
        print(f"Total parameters: {model.count_params():,}")

        # Train
        hist1, hist2 = train_model(model, base_model, name)

        # Evaluate
        results = evaluate_model(model, name, test_generator)
        all_results[name] = results

        # Save trained model
        trained_models[name] = model
        model.save(os.path.join(DRIVE_PATH, f'{name}_final.h5'))
        print(f"✅ Final model saved to Drive: {DRIVE_PATH}{name}_final.h5")


    # ========== FINAL COMPARISON ==========
    print(f"\n\n{'='*60}")
    print("FINAL MODEL COMPARISON")
    print(f"{'='*60}\n")

    import pandas as pd
    results_df = pd.DataFrame(all_results).T
    print(results_df.to_string())

    best_model = results_df['f1_score'].idxmax()
    print(f"\n🏆 Best Model: {best_model}")
    print(f"   F1-Score: {results_df.loc[best_model, 'f1_score']:.4f}")
    print(f"   AUC: {results_df.loc[best_model, 'auc']:.4f}")

    # ========== DEMO: SINGLE PREDICTION WITH ENSEMBLE ==========
    print(f"\n\n{'='*60}")
    print("DEMO: PREDICTION PIPELINE")
    print(f"{'='*60}")

    # Get a sample test image
    import os
    test_pneumonia_dir = os.path.join(TEST_DIR, 'PNEUMONIA')
    sample_images = os.listdir(test_pneumonia_dir)[:1]

    if sample_images:
        sample_path = os.path.join(test_pneumonia_dir, sample_images[0])

        # Ensemble prediction
        preds, ens_pred, ens_class, conf = ensemble_predict(trained_models, sample_path)
        print(f"\nGenerating Grad-CAM for {best_model}...")
        best_conv_layer = [layer for _, name, layer in models_config if name == best_model][0]
        visualize_gradcam(sample_path, trained_models[best_model], best_model, best_conv_layer)
print(f"\n{'='*60}")
print("✅ PIPELINE COMPLETE!")
print(f"{'='*60}")
print("\nAll models trained, evaluated, and saved.")
print("Check the generated plots and saved models in the current directory.")




############################################################
# PROCESSING: DenseNet121
############################################################

✅ Model built: DenseNet121
Total parameters: 7,337,025

TRAINING: DenseNet121

🔹 STAGE 1: Feature Extraction (base frozen)
Base model trainable: False
Epoch 1/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8204 - auc: 0.9018 - loss: 0.3719 - precision: 0.9405 - recall: 0.8065
Epoch 1: val_auc improved from -inf to 1.00000, saving model to DenseNet121_feature_extraction.h5


82/82 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.8211 - auc: 0.9024 - loss: 0.3708 - precision: 0.9408 - recall: 0.8073 - val_accuracy: 0.8750 - val_auc: 1.0000 - val_loss: 0.2667 - val_precision: 1.0000 - val_recall: 0.7500 - learning_rate: 0.0010
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9198 - auc: 0.9771 - loss: 0.1919 - precision: 0.9798 - recall: 0.9110
Epoch 2: val_auc did not improve from 1.00000
82/82 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.9198 - auc: 0.9771 - loss: 0.1917 - precision: 0.9798 - recall: 0.9111 - val_accuracy: 0.9375 - val_auc: 0.9688 - val_loss: 0.2659 - val_precision: 1.0000 - val_recall: 0.8750 - learning_rate: 0.0010
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9412 - auc: 0.9844 - loss: 0.1548 - precision: 0.9802 - recall: 0.9398
Epoch 3: val_auc did not improve from 1.00000
82/82 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.9412 - auc: 0.9844 - loss: 0.1548 - precision: 0.9802 - recall: 0.9398 - val_a

82/82 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.9444 - auc: 0.9853 - loss: 0.1508 - precision: 0.9784 - recall: 0.9458 - val_accuracy: 0.7500 - val_auc: 0.9375 - val_loss: 0.2987 - val_precision: 0.7500 - val_recall: 0.7500 - learning_rate: 1.0000e-05
Epoch 2/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9424 - auc: 0.9876 - loss: 0.1369 - precision: 0.9838 - recall: 0.9387
Epoch 2: val_auc did not improve from 0.93750
82/82 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9425 - auc: 0.9876 - loss: 0.1369 - precision: 0.9839 - recall: 0.9387 - val_accuracy: 0.7500 - val_auc: 0.9375 - val_loss: 0.3207 - val_precision: 0.7500 - val_recall: 0.7500 - learning_rate: 1.0000e-05
Epoch 3/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9457 - auc: 0.9880 - loss: 0.1332 - precision: 0.9835 - recall: 0.9429
Epoch 3: val_auc did not improve from 0.93750
82/82 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9457 - auc: 0.9880 - loss: 0.1331 - precision: 0.9835 - recall: 0.9430

82/82 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9493 - auc: 0.9900 - loss: 0.1241 - precision: 0.9838 - recall: 0.9475 - val_accuracy: 0.8125 - val_auc: 0.9688 - val_loss: 0.3139 - val_precision: 0.7778 - val_recall: 0.8750 - learning_rate: 5.0000e-06
Epoch 7/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9484 - auc: 0.9890 - loss: 0.1248 - precision: 0.9833 - recall: 0.9473
Epoch 7: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.

Epoch 7: val_auc did not improve from 0.96875
82/82 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9484 - auc: 0.9890 - loss: 0.1247 - precision: 0.9833 - recall: 0.9473 - val_accuracy: 0.8125 - val_auc: 0.9531 - val_loss: 0.3153 - val_precision: 0.7778 - val_recall: 0.8750 - learning_rate: 5.0000e-06
Epoch 8/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9561 - auc: 0.9926 - loss: 0.1032 - precision: 0.9903 - recall: 0.9497
Epoch 8: val_auc did not improve from 0.96875
82/82 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accur

✅ Model saved: DenseNet121_final.h5


############################################################
# PROCESSING: ResNet50
############################################################

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Model built: ResNet50
Total parameters: 24,153,473

TRAINING: ResNet50

🔹 STAGE 1: Feature Extraction (base frozen)
Base model trainable: False
Epoch 1/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7680 - auc: 0.8647 - loss: 0.4417 - precision: 0.9338 - recall: 0.7423
Epoch 1: val_auc improved from -inf to 0.78125, saving model to ResNet50_feature_extraction.h5


...

82/82 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - accuracy: 0.5131 - auc: 0.5106 - loss: 0.7030 - precision: 0.7580 - recall: 0.5122 - val_accuracy: 0.5000 - val_auc: 0.7266 - val_loss: 0.6935 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 2.5000e-06
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 4.

✅ EfficientNetB0 training completed!

EVALUATION: EfficientNetB0
10/10 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step

📊 Test Metrics:
  Accuracy:  0.6250
  Precision: 0.6250
  Recall:    1.0000
  F1-Score:  0.7692
  AUC:       0.5457

📋 Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.00      0.00      0.00       234
   PNEUMONIA       0.62      1.00      0.77       390

    accuracy                           0.62       624
   macro avg       0.31      0.50      0.38       624
weighted avg       0.39      0.62      0.48       624



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


✅ Model saved: EfficientNetB0_final.h5


FINAL MODEL COMPARISON

                accuracy  precision    recall  f1_score       auc
DenseNet121     0.891026   0.932796  0.889744  0.910761  0.963533
ResNet50        0.833333   0.903955  0.820513  0.860215  0.897184
EfficientNetB0  0.625000   0.625000  1.000000  0.769231  0.545726

🏆 Best Model: DenseNet121
   F1-Score: 0.9108
   AUC: 0.9635


DEMO: PREDICTION PIPELINE

ENSEMBLE PREDICTION
  DenseNet121         : 0.9291 (PNEUMONIA)
  ResNet50            : 0.9405 (PNEUMONIA)


  EfficientNetB0      : 0.5247 (PNEUMONIA)

🔮 Ensemble Result: PNEUMONIA
   Confidence: 79.81%

Generating Grad-CAM for DenseNet121...


ValueError: No such layer: conv5_block16_concat. Existing layers are: ['input_layer_3', 'densenet121', 'global_average_pooling2d_1', 'batch_normalization_1', 'dense_3', 'dropout_2', 'dense_4', 'dropout_3', 'dense_5'].

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
"""
Simple Ensemble Prediction - No Training, No LLM
Just load models and predict
"""

import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# ============================================================
# STEP 1: LOAD YOUR TRAINED MODELS
# ============================================================
print("="*60)
print("Loading Models...")
print("="*60)

# Model files (adjust names if different)
model_files = {
    'DenseNet121': '/content/drive/MyDrive/Colab Notebooks/medicalimagefiles/DenseNet121_final.h5',
    'ResNet50': '/content/drive/MyDrive/Colab Notebooks/medicalimagefiles/ResNet50_final.h5',
    'EfficientNetB0': '/content/drive/MyDrive/Colab Notebooks/medicalimagefiles/EfficientNetB0_final.h5'
}

models = {}
for name, path in model_files.items():
    if os.path.exists(path):
        print(f"Loading {name}...")
        models[name] = keras.models.load_model(path)
        print(f"  ✅ {name} loaded!")
    else:
        print(f"  ❌ {path} not found!")

print(f"\n✅ {len(models)} models loaded successfully!\n")

# ============================================================
# STEP 2: ENSEMBLE PREDICTION FUNCTION
# ============================================================
def ensemble_predict(image_path, models_dict):
    """
    Predict using ensemble of models
    """
    print(f"\n{'='*60}")
    print(f"Predicting: {image_path}")
    print(f"{'='*60}\n")

    # Load and preprocess image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Get predictions from each model
    predictions = {}
    print("Individual Predictions:")
    print("-" * 60)

    for model_name, model in models_dict.items():
        pred = model.predict(img_array, verbose=0)[0][0]
        result = "PNEUMONIA" if pred > 0.5 else "NORMAL"
        confidence = pred * 100 if pred > 0.5 else (1 - pred) * 100

        predictions[model_name] = pred
        print(f"  {model_name:20s}: {pred:.4f} → {result:10s} ({confidence:.2f}%)")

    # Calculate ensemble (average of all models)
    ensemble_score = np.mean(list(predictions.values()))
    ensemble_result = "PNEUMONIA" if ensemble_score > 0.5 else "NORMAL"
    ensemble_confidence = ensemble_score * 100 if ensemble_score > 0.5 else (1 - ensemble_score) * 100

    print("\n" + "=" * 60)
    print("ENSEMBLE RESULT:")
    print("=" * 60)
    print(f"  Prediction: {ensemble_result}")
    print(f"  Confidence: {ensemble_confidence:.2f}%")
    print(f"  Score: {ensemble_score:.4f}")
    print("=" * 60)

    return predictions, ensemble_result, ensemble_confidence

# ============================================================
# STEP 3: TEST IT!
# ============================================================

# Change this to your test image path
test_image = "/content/chest_xray/test/PNEUMONIA/person114_bacteria_546.jpeg"

# Or try multiple images
# test_images = [
#     "chest_xray/test/PNEUMONIA/person1_virus_6.jpeg",
#     "chest_xray/test/NORMAL/IM-0001-0001.jpeg",
# ]

if os.path.exists(test_image):
    predictions, result, confidence = ensemble_predict(test_image, models)
else:
    print(f"❌ Image not found: {test_image}")
    print("\n💡 To use:")
    print("  1. Change 'test_image' variable to your X-ray path")
    print("  2. Or run: ensemble_predict('your_image.jpg', models)")

Loading Models...
Loading DenseNet121...


  ✅ DenseNet121 loaded!
Loading ResNet50...


  ✅ ResNet50 loaded!
Loading EfficientNetB0...


  ✅ EfficientNetB0 loaded!

✅ 3 models loaded successfully!


Predicting: /content/chest_xray/test/PNEUMONIA/person114_bacteria_546.jpeg

Individual Predictions:
------------------------------------------------------------
  DenseNet121         : 1.0000 → PNEUMONIA  (100.00%)


  ResNet50            : 0.9695 → PNEUMONIA  (96.95%)


  EfficientNetB0      : 0.5047 → PNEUMONIA  (50.47%)

ENSEMBLE RESULT:
  Prediction: PNEUMONIA
  Confidence: 82.47%
  Score: 0.8247
